# II. Goal

## A. Ball's position

In [1]:
%matplotlib tk
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy import linalg

### Question 1

To place a virtual ball in the origin of the reference frame we first retrieve the camera matrix M from the lab

In [2]:
M=np.array([[-6.73416147e+02,  5.02373125e+02,  8.72863658e+03, -9.35587200e+04],
 [-1.14208360e+03 , 8.99624792e+03 ,-4.14476675e+02, -5.23856291e+04],
 [ 8.63643717e-01,  4.67247264e-01, -1.89207620e-01, -6.08703535e+01]]
)
print(M)

[[-6.73416147e+02  5.02373125e+02  8.72863658e+03 -9.35587200e+04]
 [-1.14208360e+03  8.99624792e+03 -4.14476675e+02 -5.23856291e+04]
 [ 8.63643717e-01  4.67247264e-01 -1.89207620e-01 -6.08703535e+01]]


Then we create our 3d point that is to say the point in the real world in homogeneous coordinate.

In [3]:
u_w = np.array([[0],[0],[0],[1]]) # origin of the reference frame in homogeneous coordiante
print(u_w)

[[0]
 [0]
 [0]
 [1]]


As in the lab we compute M times u_w to get the image position of the center of the ball in the image buffer. We don't forget to divide by the last coordinate because we are in homogeneous coordinate.

In [4]:
u_sim=np.dot(M,u_w)
u_sim = u_sim/u_sim[2,:]
print("Position x y of the center of the ball in the image buffer:")
print(u_sim[:2])

Position x y of the center of the ball in the image buffer:
[[1537.01620938]
 [ 860.60990429]]


### Question 2

Now let's draw the ball using his position in the image using scatter

In [5]:
img=mpimg.imread('goal.jpg')
plt.imshow(img)
plt.scatter(u_sim[0,:],u_sim[1,:],50)
plt.show()

The center of the reference frame is the middle of the goal line, so here the position of the ball look right.

### Question 3

Here we cannot obtain the 3D position of the real ball because we don't have any information of its position. If we had at least one the three coordinates, x, y or z we will be able to retrieve the 2 other positions by using M.

## B. Player's position

### Question 1

We want to obtain the 3D positions of the players but as we explained in the last question we need at least one of the 3 coordinates and here we have y=0. So we take the camera matrix and remove the column corresponding to y. 

In [6]:
M_prime = M[:,[0,2,3]]
print(M_prime.shape)

(3, 3)


We get the matrix M' of dimensions 3x3

### Question 2

To be able to know if the transformation is invertible we need to see if the matrix M' is invertible. So let's test

In [7]:
print(np.linalg.det(M_prime))

-1065734472.3064945


The determinant of M' is not equal to 0 so the matrix M' is invertible and the transformation is also invertible.

### Question 3

To obtain the pixel positions of one of the feet for each of the 11 players we use the code given in the lab where we had to manually click on the image to retrieve the position.

In [8]:
# load image
img=mpimg.imread('goal.jpg')
u_im= np.array([]).reshape(0,2)

Here we have commented this section and we give you u_im already fill.

In [9]:
# fig,ax = plt.subplots()
# ax.imshow(img)

# If %matplotlib works properly you can
# use the following two lines
# Image buffer points from mouse clicks
#u_im = np.array(plt.ginput(N,timeout=-1))
#plt.close()

# If you %matplotlib does not work properly
# you can try the code below
# def onclick(event):
#     global ix, iy
#     ix, iy = event.xdata, event.ydata
    
#     global u_im
#     u_im = np.vstack((u_im,np.array([[ix,iy]])))

#     if np.shape(u_im)[0] == 13:
#         fig.canvas.mpl_disconnect(cid)
#         plt.close()
        
# cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [21]:
u_im = np.array([[1165.98353408, 1091.9334918 ],
 [ 875.16314848 ,1083.17384163],
 [ 982.03088054,  999.08120002],
 [1120.4333532 ,  948.27522904],
 [1628.49306297 , 923.74820857],
 [1374.46320808 , 843.15942702],
 [ 351.33606841 , 943.01943894],
 [ 798.078227   , 780.08994581],
 [ 535.28872195 , 736.29169496],
 [ 142.85639441 , 571.6102718 ],
 [ 868.15542835 , 229.98391523]]
)
print(u_im.shape)

(11, 2)


In [22]:
# we use homogeneous coordinate
u_im = np.vstack((np.transpose(u_im),np.ones(11)))
print("Position x y of one of the feet for each of the 11 players in the image :")
print(u_im[:2])
print(u_im.shape)

Position x y of one of the feet for each of the 11 players in the image :
[[1165.98353408  875.16314848  982.03088054 1120.4333532  1628.49306297
  1374.46320808  351.33606841  798.078227    535.28872195  142.85639441
   868.15542835]
 [1091.9334918  1083.17384163  999.08120002  948.27522904  923.74820857
   843.15942702  943.01943894  780.08994581  736.29169496  571.6102718
   229.98391523]]
(3, 11)


In the next cell we plot the image with the points that we had manually selected, we can check that the points are well choosen.

In [23]:
plt.imshow(img)
plt.scatter(u_im[0,:],u_im[1,:])
plt.show()

### Question 4

To retrieve the positions in meters in the real world of the players we need to use the inverse matrix of M' and as we see in the question 2, M' is invertible.

In [27]:
M_inv = np.linalg.inv(M_prime)

u_w=np.dot(M_inv,u_im)
print(np.shape(u_w))

(3, 11)


Now we divide by the last coordinate because we are in homogeneous coordinate and we have to add in u_w the y coordinate which is 0 here, so we insert a vector of 0.

In [28]:
u_w = u_w/u_w[2,:]
u=u_w[:1]
u=np.vstack((u,np.zeros(11)))
u=np.vstack((u,u_w[1:,]))
u_w=u
print("Position of the players in the 3D space :")
print(u_w[:3])

Position of the players in the 3D space :
[[ 6.38189047e+00  5.96735024e+00  3.70676852e+00  2.32988202e+00
   2.00043381e+00 -6.98298659e-01  1.55304125e+00 -3.36776997e+00
  -5.22092157e+00 -1.23588393e+01 -2.86304423e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.72199766e+00  5.48850990e+00  4.42227826e+00  3.26386897e+00
  -1.55765813e-01  9.56256676e-01  8.37850048e+00  4.54863673e+00
   6.23403664e+00  8.56765906e+00 -3.69469877e-03]]


We can now try to plot the points in the image using u_w that we created and using the camera matrix M

In [29]:
u_sim=np.dot(M,u_w)
u_sim = u_sim/u_sim[2,:]

img=mpimg.imread('goal.jpg')
plt.imshow(img)
plt.scatter(u_sim[0,:],u_sim[1,:])
plt.show()

We see that the points corresponds to what we have manually choosen previously so the matrix M' is good.

Now let's take a look about the distances as we know the dimensions of the field.

In [30]:
alpahbet=['A','B','C','D','E','F','G','H','I','J','K']
for i in range(u_w.shape[1]):
    print("Player "+alpahbet[i]+" : position in x = "+str(np.round(u_w[0][i],2))
          +"m"+" ; in z = "+str(np.round(u_w[2][i],2))+"m")

Player A : position in x = 6.38m ; in z = 3.72m
Player B : position in x = 5.97m ; in z = 5.49m
Player C : position in x = 3.71m ; in z = 4.42m
Player D : position in x = 2.33m ; in z = 3.26m
Player E : position in x = 2.0m ; in z = -0.16m
Player F : position in x = -0.7m ; in z = 0.96m
Player G : position in x = 1.55m ; in z = 8.38m
Player H : position in x = -3.37m ; in z = 4.55m
Player I : position in x = -5.22m ; in z = 6.23m
Player J : position in x = -12.36m ; in z = 8.57m
Player K : position in x = -28.63m ; in z = -0.0m


The distances seem adequate with the dimensions of the field. If we take for example the player K, we see that we have 0m in z axis which is the case. In the x axis we have -28m and the corner is at 32.5m of the center of the goal line. So the player K who is close to the corner has a good estimation. Here it's an example but we can look for the other players and see that we have reasonable values in meter according to the dimension of the field.